## Импорт библиотек и модулей:
### Импортируются необходимые библиотеки и модули, такие как os, sys, email_read_util и различные классы из pyspark.


In [ ]:
import os
import sys
sys.path.append('../chapter1')
import email_read_util

from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

## Подготовка данных:
### Загружается набор данных "2007 TREC Public Spam Corpus" и распаковывается в директорию 'chapter1/datasets'. Проверяются пути к директориям 'data_dir' и 'labels_path'.

In [ ]:
DATA_DIR = '../chapter1/datasets/trec07p/data/'
LABELS_FILE = '../chapter1/datasets/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

### Чтение меток (labels):
Читаются метки из файла 'index' и сохраняются в словаре labels, где ключ - это имя файла, а значение - метка (1 для 'ham' и 0 для других).

In [ ]:
labels = {}
# Read the labels.
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

### Чтение электронных писем:
Функция read_email_files() считывает текст электронных писем и их метки, используя утилиту email_read_util.

In [ ]:
def read_email_files():
    X = []
    y = [] 
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = email_read_util.extract_email_text(
            os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

### Создается DataFrame df с полями 'id', 'email' и 'label' на основе считанных данных.

In [ ]:
X, y = read_email_files()

schema = StructType([
            StructField('id', IntegerType(), nullable=False),
            StructField('email', StringType(), nullable=False),
            StructField('label', DoubleType(), nullable=False)])

df = spark.createDataFrame(zip(range(len(y)), X, y), schema)

In [ ]:
df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- email: string (nullable = false)
 |-- label: double (nullable = false)

### DataFrame разделяется на обучающую и тестовую выборки в соотношении 70% к 30%.

In [ ]:
train, test = df.randomSplit([TRAINING_SET_RATIO, 1-TRAINING_SET_RATIO], seed=123)

### Построение модели:
#### Создается конвейер pipeline, включающий токенизатор, векторизатор и классификатор случайного леса.
#### Устанавливаются параметры для каждого этапа конвейера.
#### Модель обучается на обучающей выборке с установленными параметрами.


In [ ]:
tokenizer = Tokenizer()
vectorizer = CountVectorizer()
rfc = RandomForestClassifier()

pipeline = Pipeline(stages=[tokenizer, vectorizer, rfc])

In [ ]:
paramMap = {
    tokenizer.inputCol: 'email',
    tokenizer.outputCol: 'tokens',

    vectorizer.inputCol: 'tokens',
    vectorizer.outputCol: 'vectors',

    rfc.featuresCol: 'vectors',
    rfc.labelCol: 'label',
    rfc.numTrees: 500
}

In [ ]:
model = pipeline.fit(train, params=paramMap)

In [ ]:
prediction = model.transform(test)

### Оценка модели:
#### Делаются предсказания на тестовой выборке с помощью обученной модели.
#### Вычисляются метрики оценки модели: площадь под ROC-кривой и площадь под кривой precision/recall.
#### Выводятся значения метрик: "Area under ROC curve score" и "Area under precision/recall curve score".

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
pr_score = evaluator.evaluate(prediction, {evaluator.metricName: 'areaUnderPR'})
roc_score = evaluator.evaluate(prediction, {evaluator.metricName: 'areaUnderROC'})

print("Area under ROC curve score: {:.3f}".format(roc_score))
print("Area under precision/recall curve score: {:.3f}".format(pr_score))

Area under ROC curve score: 0.971
Area under precision/recall curve score: 0.958


### Результаты:
#### Area under ROC curve score: 0.971
#### Area under precision/recall curve score: 0.958


In [ ]:
Эти результаты представляют собой оценки производительности модели классификации на основе метрик ROC-кривой и кривой точности/полноты.

1. **Площадь под ROC-кривой (Area under ROC curve)**:
   - ROC-кривая показывает отношение между долей верных положительных и долей ложных положительных классификаций при варьировании порога для классификации.
   - Площадь под ROC-кривой (Area under ROC curve, AUC-ROC) является мерой качества классификатора, при которой оценивается его способность различать между классами. Чем ближе AUC-ROC к 1, тем лучше производительность модели. Значение 0.971 указывает на высокий уровень производительности модели.

2. **Площадь под кривой точности/полноты (Area under precision/recall curve)**:
   - Кривая точности/полноты (Precision/Recall curve) показывает отношение между точностью (Precision) и полнотой (Recall) классификатора при различных порогах классификации.
   - Площадь под этой кривой (Area under precision/recall curve, AUC-PR) также является мерой качества модели. Чем ближе AUC-PR к 1, тем лучше производительность модели. Значение 0.958 указывает на высокий уровень производительности модели по этой метрике.

Таким образом, эти результаты свидетельствуют о том, что модель имеет хорошие характеристики в отношении как различения между классами, так и точности и полноты предсказаний.